# Example to load SWOT dataset (netcdf)

In [2]:
import s3fs
import sys
from itertools import product
import xarray as xr
import intake
import yaml
import netCDF4 



## Creating catalog for S3/netcdf files

In [ ]:

cycle_number=510
base_s3_folder= "s3://project-meom-ige/SWOT_L3_V1_0"
s3_folder = f"{base_s3_folder}/cycle_{cycle_number}/"
endpoint_url= 'https://minio.lab.dive.edito.eu/'
s3 = s3fs.S3FileSystem(anon=True,client_kwargs={"endpoint_url": endpoint_url})

# List NetCDF files in the folder
netcdf_files = [(endpoint_url+file) for file in s3.glob(f"{base_s3_folder}/cycle_{cycle_number}/*.nc")]

if not netcdf_files:
    raise ValueError("No NetCDF files found in the specified S3 location.")


# 🔹 Generate Intake catalog YAML with correct `storage_options`
#intake_xarray.NetCDFSource
catalog_dict = {
    "sources": {
        "SWOT": {
            "description": "SWOT dataset load ",
            "driver": "netcdf",
            "args": {
                "urlpath": netcdf_files,  # Explicit list of S3 URLs
                "engine": "h5netcdf",
                "chunks": {}
            }
        }
    }
}



# Save YAML catalog
with open("testcatalog.yaml", "w") as yaml_file:
    yaml.dump(catalog_dict, yaml_file, default_flow_style=False)

## Load Catalog

In [3]:

cat = intake.open_catalog("testcatalog.yaml")
raw_entry = cat._entries['SWOT'].describe()
url_list = raw_entry["args"]["urlpath"]
single_url = url_list

# To create one xarray, the files have to concadenate to a dimension. This case num_lines 
new_source = intake.open_netcdf(urlpath=single_url,xarray_kwargs={'drop_variables': ['i_num_line','i_num_pixel']},engine="h5netcdf",combine="nested",concat_dim="num_lines")
ds0 = new_source.read()  # or .to_dask() if it's large

In [4]:
ds0

<xarray.Dataset> Size: 764MB
Dimensions:         (num_lines: 276080, num_pixels: 69)
Coordinates:
    latitude        (num_lines, num_pixels) float64 152MB dask.array<chunksize=(9860, 69), meta=np.ndarray>
    longitude       (num_lines, num_pixels) float64 152MB dask.array<chunksize=(9860, 69), meta=np.ndarray>
Dimensions without coordinates: num_lines, num_pixels
Data variables:
    time            (num_lines) datetime64[ns] 2MB dask.array<chunksize=(9860,), meta=np.ndarray>
    mdt             (num_lines, num_pixels) float64 152MB dask.array<chunksize=(9860, 69), meta=np.ndarray>
    ssha            (num_lines, num_pixels) float64 152MB dask.array<chunksize=(9860, 69), meta=np.ndarray>
    ssha_noiseless  (num_lines, num_pixels) float64 152MB dask.array<chunksize=(9860, 69), meta=np.ndarray>
Attributes: (12/42)
    Conventions:                     CF-1.7
    Metadata_Conventions:            Unidata Dataset Discovery v1.0
    cdm_data_type:                   Swath
    comment:                         Sea Surface Height measured by Altimetry
    geospatial_lat_units:            degrees_north
    geospatial_lon_units:            degrees_east
    ...                              ...
    geospatial_lat_min:              -78.272196
    geospatial_lat_max:              78.272247
    geospatial_lon_min:              8.4e-05
    geospatial_lon_max:              359.999504
    data_used:                       SWOT KaRIn L2_LR_SSH PGC0 (NASA/CNES). D...
    doi:                             https://doi.org/10.24400/527896/A01-2023...